In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data import Field, TabularDataset, BucketIterator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import re
import numpy as np
import pandas as pd
import random
import math
import time
from tqdm import tqdm_notebook as tqdm
import re

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda')

In [ ]:
torch.__version__

In [3]:
print(torch.version.cuda)

10.2


# Data Stopword & Nonalphanumeric Removal, Splitting

In [8]:
def read_txt_one(path, store_dict, strornot):
    with open(path,'r',encoding='UTF-8') as file :
        for line in tqdm(file) :
            new = line.strip()
            if not strornot:
                store_dict[int(new)] = 'y'
            else:
                store_dict[new] = 'y'

In [9]:
stopwords = {}
read_txt_one('./tra_chn_stpwords.txt', stopwords, True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
def remove_stopwords(row):
    new_row = []
    for word in row.split(' '):
        try:
            stopwords[word]
        except:
            new_row.append(word)        
    return ' '.join(new_row) 

In [11]:
def preserve_alphanumeric(row):
    rule = re.compile(r"[^a-zA-Z0-9\u4e00-\u9fa5]")
    row = rule.sub(' ',row)
    row = ' '.join([word for word in row.split(' ') if word])
    return row

In [12]:
data = pd.read_csv('udn_train.csv', encoding = 'utf-8')[['headline_tk', 'body_tk']]

In [33]:
data.head()

,headline_tk,body_tk,hl_tok_stp,bd_tok_stp
0,不 怕 高 持有稅 ， 囤房族 增加 逾 5千 人 擁有 逾 10 戶,各 縣 市政府 近年 陸續 調高 不 動產 稅基 ， 使地 價稅 及 房屋 稅 增加 ， 讓...,高 持有稅 囤房族 增加 逾 5千 人 擁有 逾 10 戶,高 持有稅 囤房族 增加 逾 5千 人 擁有 逾 10 戶
1,MLB／ 誰 是 胡智為 ？ ， 光芒 貼心教 發音,升上 大 聯盟 後 的 台灣 小 將 胡 智為 26 日 作客 巴爾 的 摩金 鶯隊 ， 仍...,MLB 胡智為 光芒 貼心教 發音,MLB 胡智為 光芒 貼心教 發音
2,女神 陳綺貞 面前 聊 A 的 ！ 五月天 差點 變 「 五月本 」,"( function ( d , s , id ) { var js ,...",女神 陳綺貞 面前 聊 五月天 差點 變 五月本,女神 陳綺貞 面前 聊 五月天 差點 變 五月本
3,馬來西亞 前 首相 納吉 被 控 洗錢 ， 出庭 不 認罪,馬來 西亞 前首相 納吉 （ Najib Razak ） 涉數 十億 美元 金融 醜聞 ...,馬來西亞 前 首相 納吉 控 洗錢 出庭 認罪,馬來西亞 前 首相 納吉 控 洗錢 出庭 認罪
4,紙風車 待用券 ， 邀 新 住民 欣賞 《 諸葛 》,魔鬼 黨們 後 空翻 入場 ， 搭配 動感 的 主題 曲音樂 ， 在 舞台 上 手舞足蹈 ，...,紙風車 待用券 邀 新 住民 欣賞 諸葛,紙風車 待用券 邀 新 住民 欣賞 諸葛


In [13]:
data['hl_tok_stp']=data['headline_tk'].apply(preserve_alphanumeric)
data['bd_tok_stp']=data['body_tk'].apply(preserve_alphanumeric)
data['hl_tok_stp']=data['hl_tok_stp'].apply(remove_stopwords)
data['bd_tok_stp']=data['bd_tok_stp'].apply(remove_stopwords)

In [16]:
valid = np.random.choice([i for i in range(len(data))],size = 49297, replace = False)
train = [i for i in range(len(data)) if i not in valid]

In [19]:
data.loc[train,:].to_csv('udn_train_new.csv', index = False)
data.loc[valid,:].to_csv('udn_valid_new.csv', index = False)

In [24]:
data = pd.read_csv('udn_test.csv', encoding = 'utf-8')[['headline_tk', 'body_tk']]

In [25]:
data['hl_tok_stp']=data['headline_tk'].apply(preserve_alphanumeric)
data['bd_tok_stp']=data['body_tk'].apply(preserve_alphanumeric)
data['hl_tok_stp']=data['hl_tok_stp'].apply(remove_stopwords)
data['bd_tok_stp']=data['bd_tok_stp'].apply(remove_stopwords)

In [27]:
data.to_csv('udn_test_new.csv', index = False)

# Character Based

In [33]:
data_train = pd.read_csv('udn_train_new.csv', encoding = 'utf-8')[['hl_tok_stp', 'bd_tok_stp']]
data_valid = pd.read_csv('udn_valid_new.csv', encoding = 'utf-8')[['hl_tok_stp', 'bd_tok_stp']]
data_test = pd.read_csv('udn_test_new.csv', encoding = 'utf-8')[['hl_tok_stp', 'bd_tok_stp']]

In [34]:
def char_based(row):
    try:
        return ' '.join(list(''.join(row.split(' '))))
    except:
        return row

In [35]:
data_train['hl_tok_stp'] = data_train['hl_tok_stp'].apply(char_based)
data_train['bd_tok_stp'] = data_train['bd_tok_stp'].apply(char_based)

data_valid['hl_tok_stp'] = data_valid['hl_tok_stp'].apply(char_based)
data_valid['bd_tok_stp'] = data_valid['bd_tok_stp'].apply(char_based)

data_test['hl_tok_stp'] = data_test['hl_tok_stp'].apply(char_based)
data_test['bd_tok_stp'] = data_test['bd_tok_stp'].apply(char_based)

In [36]:
data_train_new.to_csv('udn_train_char.csv', index = False)
data_valid.to_csv('udn_valid_char.csv', index = False)
data_test.to_csv('udn_test_char.csv', index = False)

In [47]:
del data_train, data_valid, data_test

# Cut news to 400 characters ( median length)

In [11]:
data_train = pd.read_csv('udn_train_char.csv', encoding = 'utf-8')
data_valid = pd.read_csv('udn_valid_char.csv', encoding = 'utf-8')
data_test = pd.read_csv('udn_test_new.csv', encoding = 'utf-8')

In [18]:
def cut_to_median(row):
    return str(row)[:800]

In [19]:
data_train['hl_tok_stp'] = data_train['hl_tok_stp'].apply(cut_to_median)
data_train['bd_tok_stp'] = data_train['bd_tok_stp'].apply(cut_to_median)

In [20]:
data_valid['hl_tok_stp'] = data_valid['hl_tok_stp'].apply(cut_to_median)
data_valid['bd_tok_stp'] = data_valid['bd_tok_stp'].apply(cut_to_median)

In [21]:
data_test['hl_tok_stp'] = data_test['hl_tok_stp'].apply(cut_to_median)
data_test['bd_tok_stp'] = data_test['bd_tok_stp'].apply(cut_to_median)

In [22]:
data_train.to_csv('udn_train_char_med.csv', index = False)
data_valid.to_csv('udn_valid_char_med.csv', index = False)
data_test.to_csv('udn_test_char_med.csv', index = False)

In [36]:
len(data_train.loc[5,:]['bd_tok_stp'])

800

# Prepare Data

In [53]:
tokenize = lambda x: x.split(' ')
SRC = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>')

TRG = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>')

train_data, valid_data, test_data = TabularDataset.splits(
                                    path = './',
                                    train = 'udn_train_char.csv',
                                    validation = 'udn_valid_char.csv',
                                    test = 'udn_test_char.csv',
                                    format = 'csv',
                                    fields = (('trg', TRG), ('src',SRC)),
                                    skip_header = True)

SRC.build_vocab(train_data, min_freq = 0)
TRG.build_vocab(train_data, min_freq = 0)

print(f"Unique tokens in source vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target vocabulary: {len(TRG.vocab)}")
BATCH_SIZE = 16
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device, 
     sort_within_batch = True,
     sort_key = lambda x : len(x.src))
     
del train_data, valid_data
torch.cuda.empty_cache()

In [54]:
SRC = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>')

TRG = Field(tokenize = tokenize, 
            init_token = '<sos>', 
            eos_token = '<eos>')

In [57]:
train_data, valid_data, test_data = TabularDataset.splits(
                                    path = './',
                                    train = 'udn_train_char.csv',
                                    validation = 'udn_valid_char.csv',
                                    test = 'udn_test_char.csv',
                                    format = 'csv',
                                    fields = (('trg', TRG), ('src',SRC)),
                                    skip_header = True)

In [58]:
SRC.build_vocab(train_data, min_freq = 0)
TRG.build_vocab(train_data, min_freq = 0)

In [59]:
print(f"Unique tokens in source vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target vocabulary: {len(TRG.vocab)}")

Unique tokens in source vocabulary: 9232
Unique tokens in target vocabulary: 5967


In [146]:
BATCH_SIZE = 32
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device, 
     sort_within_batch = True,
     sort_key = lambda x : len(x.src))

In [60]:
del train_data, valid_data

In [12]:
torch.cuda.empty_cache()

# Building the Seq2Seq Model

## Encoder

In [132]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n direction, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]    
        #outputs are always from the top hidden layer
        
        return hidden, cell
    
    def initHidden():
        return torch.zeros(1, self.hid_dim, self.hid_dim, device=device)

## Decoder

In [133]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.softmax(self.fc_out(output.squeeze(0)))
        
        #prediction = [batch size, output dim]
        
        
        return prediction, hidden, cell

## Seq2Seq 

In [135]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        encoder_hidden, encoder_cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        decoder_input = trg[0,:]
        
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, decoder_hidden, decoder_cell = self.decoder(decoder_input, decoder_hidden, decoder_cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            decoder_input = trg[t] if teacher_force else top1
        
        return outputs

# Training the Seq2Seq Model 

In [149]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [96]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(11229, 256)
    (rnn): LSTM(256, 512, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(6205, 256)
    (rnn): LSTM(256, 512, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=6205, bias=True)
    (softmax): LogSoftmax()
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [97]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,800,189 trainable parameters


In [225]:
for i,p in enumerate(model.parameters()):
    print(p.data)
    p.data

tensor([[-0.0325,  0.0669,  0.0255,  ...,  0.0019, -0.0480, -0.0583],
        [ 0.0754,  0.0688, -0.0305,  ...,  0.0767, -0.0490,  0.0415],
        [-0.0789,  0.0481, -0.0205,  ...,  0.0798,  0.0528, -0.0080],
        ...,
        [-0.0684, -0.0038, -0.0237,  ..., -0.0545,  0.0070, -0.0106],
        [ 0.0194,  0.0226, -0.0183,  ..., -0.0392, -0.0684,  0.0670],
        [-0.0099, -0.0335,  0.0674,  ..., -0.0003, -0.0179,  0.0753]],
       device='cuda:0')
tensor([[ 6.5498e-05, -4.3551e-04,  6.1081e-02,  ...,  5.8089e-02,
          2.4228e-02,  3.2461e-02],
        [-6.2883e-02, -3.2338e-03,  1.5267e-02,  ...,  3.1488e-02,
          1.0186e-02, -7.8064e-02],
        [-4.0181e-02, -1.8052e-02, -5.7386e-03,  ..., -2.6327e-02,
          7.0339e-02,  3.5711e-02],
        ...,
        [ 2.0285e-03,  2.1925e-02,  6.7439e-02,  ..., -8.5020e-04,
          2.4513e-02, -6.8367e-02],
        [-7.4410e-02,  7.6930e-02,  6.1950e-02,  ..., -1.7178e-02,
          2.2511e-02, -8.9298e-03],
        [ 7.11

In [137]:
optimizer = optim.Adam(model.parameters(), lr = 1e-04)  

In [138]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [139]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        output = model(src, trg)
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [140]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg, 0) #turn off teacher forcing
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            epoch_loss += loss.item()  
            
    return epoch_loss / len(iterator)

In [141]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 30
CLIP = 1

best_valid_loss = float('inf')
last_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model_med.pt')
        
    if last_valid_loss <= valid_loss:
        break
        
    last_valid_loss = valid_loss    
    
    print(f'Epoch: {epoch+1:2} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



Epoch:  1 | Time: 17m 30s
	Train Loss: 8.733 | Train PPL: 6204.231
	 Val. Loss: 8.731 |  Val. PPL: 6189.712


In [185]:
model.load_state_dict(torch.load('tut1-model.pt',  map_location=torch.device("cuda")),strict=False)
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(586779, 64)
    (rnn): LSTM(64, 64, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(70797, 64)
    (rnn): LSTM(64, 64, dropout=0.5)
    (fc_out): Linear(in_features=64, out_features=70797, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [1]:
def summarize_sentence(sentence, src_field, trg_field, model, device, max_len = 30):
    model.eval()
    tokens = [src_field.init_token] + sentence + [src_field.eos_token]
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    #print(src_indexes)
    src_tensor = torch.tensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        trg_tensor = torch.tensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    if trg_tokens[-1] == trg_field.eos_token : 
        trg_tokens = trg_tokens[:-1]

    return trg_tokens[1:]

In [72]:
example_idx = 100

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['端午', '節', '將', '至', '，', '不少', '糖尿病', '友會', '詢問', '營養師', '能否', '吃', '粽子', '。', '答案', '是', '肯定', '的', '，', '只要', '謹記', '適量', '攝取', '且', '把握', '低油', '、', '低鹽', '、', '低糖', '、', '高纖', '與', '均衡', '飲食', '的', '原', '則', '，', '仍', '可', '和', '家人', '共享', '端午', '美食', '與', '佳節', '氣氛', '。', '童綜合', '醫院', '營養', '治療', '科營', '養師', '黃筱尹', '表示', '，', '傳統', '粽子', '主要', '以', '糯米', '為主', '材料', '再', '搭配', '其他', '食材', '（', '如', '栗子', '、', '蛋黃', '、', '滷肉', '、', '花生', '等', '）', '一同', '包覆', '而成', '，', '而', '糯米', '屬全', '榖根', '莖類', '食物', '，', '因此', '需注意', '食用', '的', '份量', '，', '否則', '易使', '血糖', '上升', '。', '一', '顆', '鹹', '粽約', '含', '一碗', '飯', '，', '鹼粽', '則', '有', '半碗', '飯', '，', '食用', '時', '可', '選擇', '中型', '粽子', '，', '或', '將', '其', '納入', '飲食', '計畫', '中', '並', '與', '米飯', '做', '代換', '，', '這', '樣可減', '少額', '外醣類', '攝取', '。', '近年', '養生', '風氣', '盛行', '，', '業者', '紛紛', '推出', '雜糧養', '生粽', '、', '健康', '全', '榖粽', '等', '，', '製程', '上', '強調', '少油', '、', '少鹽', '與', '高纖', '，', '這', '對', '於', '糖尿病', '友是', '另', '一', '項', '選擇',

In [73]:
summarization = summarize_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {summarization}')

15693
10848
10848
10848
10848
10848
10848
57112
57112
57112
57112
57112
57112
44845
10848
predicted trg = ['場館', '辯護', '辯護', '辯護', '辯護', '辯護', '辯護', '餡料', '餡料', '餡料', '餡料', '餡料', '餡料', '公布稅', '辯護']


In [ ]:
file = open('system_summaries_seq2seq.txt', 'w', encoding = 'utf-8')
for i in range(49297):
    src = vars(test_data.examples[i])['src']
    #trg = vars(test_data.examples[i])['trg']
    file.write(''.join(summarize_sentence(src, SRC, TRG, model, device)))
    file.write('\n')
    if i % 1000 == 0:
        print(i)
file.close()    